In [1]:
import torch 
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
from utility import (
    load_object
)


In [11]:
fileName = 'results/surrogate_model_11_28_59_26_04_2024'

In [23]:
# Defne a NN model
class Net(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        super(Net, self).__init__()
        self.fc1 = nn.Linear(self.input_size, self.hidden_size)
        self.fc2 = nn.Linear(self.hidden_size, self.hidden_size)
        self.fc3 = nn.Linear(self.hidden_size, self.hidden_size)
        self.fc4 = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = self.fc4(x)
        return x

In [33]:
parameters_list = load_object(fileName + "/Data", "param_values")
returns_timeseries = load_object(fileName + "/Data", "returns_timeseries")

input_size = len(returns_timeseries[0])
hidden_size = 50
output_size = len(parameters_list[0])


In [34]:

#Prepare the data for the neural network
X = torch.tensor(np.asarray(returns_timeseries)).float()
y = torch.tensor(parameters_list).float()

In [35]:
#import the NN model from the Data file in pt format
model = Net(input_size, hidden_size, output_size)
model.load_state_dict(torch.load(fileName + "/Data/NN_model.pt"))

<All keys matched successfully>

In [36]:
#evaluate the model on X[0]
model(X[0]), y[0]

(tensor([ 0.0948, -0.3691,  0.6803,  0.4851,  0.1591,  0.1688,  0.7716],
        grad_fn=<ViewBackward0>),
 tensor([ 0.1223, -0.3359,  0.6566,  0.4637,  0.2266,  0.1722,  0.7838]))